In [1]:
import os
from datetime import datetime, timedelta, timezone
import shutil

import numpy as np
import pandas as pd
import xarray as xr

In [2]:
from configs.config_object import ConfigObject
from grid_and_bathy import build_and_save_mitgcm_grid, get_dz_grid
from surface_forcings import extract_and_save_surface_forcings
from initial_conditions import download_profile_idronaut_datalakes, create_initial_temperature_from_measure_profile
from run_preprocessing import write_data_config_files, copy_template, remove_all_files_and_folders, write_size_config_files
from utils import modify_arguments, convert_binary_files

### Load config

In [3]:
config = ConfigObject('configs/config.json')
output_folder = './99-output_preprocessing/'
template_folder = './00-template_mitgcm/'

In [4]:
parsed_start_date = datetime.strptime(config.start_date, '%Y%m%d')
parsed_end_date = datetime.strptime(config.end_date, '%Y%m%d')
sim_duration_in_second = (parsed_end_date - parsed_start_date).total_seconds()

### Initialize output folder

In [5]:
remove_all_files_and_folders(output_folder)
copy_template(template_folder, output_folder)
config.write_metadata_to_file(os.path.join(output_folder, 'metadata_simulation.txt'))

### Build MITgcm grid

build_and_save_mitgcm_grid(config.paths.grid_folder_path, 
                           config.Nx, 
                           config.Ny, 
                           config.grid_resolution, 
                           config.x0_ch1903, 
                           config.y0_ch1903, 
                           config.x1_ch1903, 
                           config.y1_ch1903)

### Build binary files

In [5]:
binary_data_folder = os.path.join(output_folder, 'binary_data')
if not os.path.exists(binary_data_folder):
    os.makedirs(binary_data_folder)
    print(f"Directory '{binary_data_folder}' created.")

**Get bathymetry**

In [6]:
#TO DO : create bathymetry
shutil.copy(config.paths.bathy_path, os.path.join(binary_data_folder, 'bathy.bin'))

'./output_preprocessing/binary_data\\bathy.bin'

**Get surface forcings**

In [18]:
extract_and_save_surface_forcings(binary_data_folder, 
                                  config.start_date, 
                                  config.end_date, 
                                  config.paths.raw_results_from_api_folder, 
                                  config.paths.grid_folder_path)

**Convert to little endian if necessary** (for cscs computation)

In [7]:
# TO DO : implement check to see whether the datatypes are correct
if config.endian_type == 'little_endian':
    convert_binary_files(os.path.join(output_folder, 'binary_data'), '>f8', '<f8')

### Get initial conditions

In [10]:
file_date, measured_temperature = download_profile_idronaut_datalakes(parsed_start_date)
print(file_date)
dz_grid = get_dz_grid(config.paths.dz_grid_csv_path)
shaped_temp_initial = create_initial_temperature_from_measure_profile(dz_grid, measured_temperature)

2024-07-31 02:56:31.500000


### Adapt config files

In [11]:
modify_arguments('!start_date!', [config.start_date], os.path.join(output_folder, 'run/data.cal'))

In [12]:
modify_arguments('!start_date!', [config.start_date], os.path.join(output_folder, 'run/data.exf'))

In [13]:
write_data_config_files(os.path.join(output_folder, 'run/data'), shaped_temp_initial, sim_duration_in_second, dz_grid, config.grid_resolution)

In [14]:
write_size_config_files(os.path.join(output_folder, 'code/SIZE.h'), 
                        config.Px, 
                        config.Py, 
                        config.Nx, 
                        config.Ny, 
                        len(dz_grid.flatten()))

### TO DO

In [14]:
# add automatic change of pickup file
# add automatic change of computing time? (submit-daint)